# GitHub Integration Test Notebook

This notebook tests the GitHub integration functionality including:
- Authentication
- Repository operations
- Branch management
- Pull request operations
- Tag management
- Connection validation

In [3]:
# Setup and imports
import os
import asyncio
import json
from typing import Dict, Any
from IPython.display import display, HTML

# Change to backend directory
os.chdir("../../../backend/")
print(f"Current working directory: {os.getcwd()}")

Current working directory: c:\Users\zakir.hussain\Documents\zakir\projects\hackathon\ProjectEnigma\backend


In [4]:
# Import GitHub integration components
from app.integrations.factory import get_api_factory
from app.integrations.base.github_interface import (
    GitHubRepository, GitHubBranch, GitHubPullRequest, GitHubTag
)
from app.core.config import get_settings

print("GitHub integration components imported successfully")
settings = get_settings()
settings.github_organization

GitHub integration components imported successfully


'ProjectEnigmaHackathon'

In [5]:
# Initialize API factory and get GitHub client
factory = get_api_factory()
github_client = factory.create_client("github", use_mock=True)  # Use mock for testing

print(f"GitHub client created: {type(github_client).__name__}")
print(f"Using mock implementation: {hasattr(github_client, '_mock_data')}")

GitHub client created: MockGitHubClient
Using mock implementation: False


## 1. Authentication Test

In [6]:
# Test authentication
async def test_authentication():
    try:
        auth_result = await github_client.authenticate()
        print(f"Authentication result: {auth_result}")
        return auth_result
    except Exception as e:
        print(f"Authentication error: {e}")
        return False

# Run authentication test
auth_success = await test_authentication()
print(f"✅ Authentication test {'PASSED' if auth_success else 'FAILED'}")

Authentication result: True
✅ Authentication test PASSED


## 2. Connection Validation Test

In [7]:
# Test connection validation
async def test_connection_validation():
    try:
        connection_info = await github_client.validate_connection()
        print("Connection validation result:")
        print(json.dumps(connection_info, indent=2))
        return connection_info
    except Exception as e:
        print(f"Connection validation error: {e}")
        return None

# Run connection validation test
connection_info = await test_connection_validation()
print(f"✅ Connection validation test {'PASSED' if connection_info else 'FAILED'}")

Connection validation result:
{
  "status": "connected",
  "user": "mock-user",
  "login": "mock-user",
  "name": "Mock User",
  "email": "mock-user@company.com",
  "organizations": [
    "ProjectEnigmaHackathon"
  ]
}
✅ Connection validation test PASSED


## 3. Repository Operations Test

In [8]:
# Test repository operations
async def test_repository_operations():
    test_repo = "test-owner/test-repo"
    
    try:
        # Get repository information
        repo_info = await github_client.get_repository(test_repo)
        print(f"Repository info: {repo_info}")
        
        # Get branches
        branches = await github_client.get_branches(test_repo)
        print(f"Found {len(branches)} branches")
        for branch in branches:
            print(f"  - {branch.name} (SHA: {branch.sha[:8]}...)")
        
        return True
    except Exception as e:
        print(f"Repository operations error: {e}")
        return False

# Run repository operations test
repo_test_success = await test_repository_operations()
print(f"✅ Repository operations test {'PASSED' if repo_test_success else 'FAILED'}")

Repository info: None
Found 0 branches
✅ Repository operations test PASSED


## 4. Branch Management Test

In [9]:
# Test branch management
async def test_branch_management():
    test_repo = "test-owner/test-repo"
    ticket_ids = ["123", "456", "789"]
    try:
        # Find feature branches
        feature_branches = await github_client.find_feature_branches(test_repo,ticket_ids)
        print(f"Feature branches found: {len(feature_branches)}")
        for branch_name, branch_info in feature_branches.items():
            if branch_info:
                print(f"  - {branch_name}: {branch_info.sha[:8]}...")
        
        # Create a new branch
        new_branch = await github_client.create_branch(
            test_repo, "feature/test-branch", "main"
        )
        print(f"Created new branch: {new_branch.name}")
        
        return True
    except Exception as e:
        print(f"Branch management error: {e}")
        return False

# Run branch management test
branch_test_success = await test_branch_management()
print(f"✅ Branch management test {'PASSED' if branch_test_success else 'FAILED'}")

Feature branches found: 3
Created new branch: feature/test-branch
✅ Branch management test PASSED


## 5. Pull Request Operations Test

In [10]:
# Test pull request operations
async def test_pull_request_operations():
    test_repo = "test-owner/test-repo"
    
    try:
        # Check merge status
        merge_status = await github_client.check_merge_status(
            test_repo, "feature/test-branch", "main"
        )
        print(f"Merge status: {merge_status}")
        
        # Create a pull request
        pr = await github_client.create_pull_request(
            test_repo,
            "Test PR Title",
            "Test PR Description",
            "feature/test-branch",
            "main"
        )
        print(f"Created PR #{pr.number}: {pr.title}")
        
        # Merge branches
        merge_result = await github_client.merge_branches(
            test_repo, "feature/test-branch", "main"
        )
        print(f"Merge result: {merge_result}")
        
        return True
    except Exception as e:
        print(f"Pull request operations error: {e}")
        return False

# Run pull request operations test
pr_test_success = await test_pull_request_operations()
print(f"✅ Pull request operations test {'PASSED' if pr_test_success else 'FAILED'}")

Merge status: {'merged': False, 'merge_commit_sha': None, 'ahead_by': 0, 'behind_by': 0}
Created PR #1: Test PR Title
Merge result: {'merged': True, 'sha': 'fedcba0987654321fedcba0987654321fedcba09', 'message': 'Merged feature/test-branch into main'}
✅ Pull request operations test PASSED


## 6. Tag Management Test

In [ ]:
# Test tag management
async def test_tag_management():
    test_repo = "test-owner/test-repo"
    
    try:
        # Get existing tags
        tags = await github_client.get_tags(test_repo)
        print(f"Found {len(tags)} tags")
        for tag in tags:
            print(f"  - {tag.name}: {tag.sha[:8]}...")
        
        # Create a new tag
        new_tag = await github_client.create_tag(
            test_repo, "v1.0.0", "abc123def456", "Release v1.0.0"
        )
        print(f"Created tag: {new_tag.name}")
        
        return True
    except Exception as e:
        print(f"Tag management error: {e}")
        return False

# Run tag management test
tag_test_success = await test_tag_management()
print(f"✅ Tag management test {'PASSED' if tag_test_success else 'FAILED'}")

## 7. Real vs Mock Comparison Test

In [ ]:
# Compare mock vs real implementations
async def compare_mock_vs_real():
    print("=== Mock Implementation ===")
    mock_client = factory.create_client("github", use_mock=True)
    mock_auth = await mock_client.authenticate()
    print(f"Mock auth: {mock_auth}")
    
    print("\n=== Real Implementation ===")
    try:
        real_client = factory.create_client("github", use_mock=False)
        real_auth = await real_client.authenticate()
        print(f"Real auth: {real_auth}")
    except Exception as e:
        print(f"Real implementation error (expected if no real credentials): {e}")

# Run comparison test
await compare_mock_vs_real()

## 8. Error Handling Test

In [ ]:
# Test error handling
async def test_error_handling():
    print("Testing error handling scenarios...")
    
    # Test with invalid repository
    try:
        await github_client.get_repository("invalid/repo")
    except Exception as e:
        print(f"✅ Expected error for invalid repo: {type(e).__name__}")
    
    # Test with invalid branch
    try:
        await github_client.get_branches("test-owner/test-repo")
    except Exception as e:
        print(f"✅ Expected error for invalid branch: {type(e).__name__}")
    
    print("Error handling tests completed")

In [ ]:
# Run error handling test
await test_error_handling()

## 9. Summary and Results

In [ ]:
# Display test summary
def display_test_summary():
    summary = {
        "Authentication": auth_success,
        "Connection Validation": bool(connection_info),
        "Repository Operations": repo_test_success,
        "Branch Management": branch_test_success,
        "Pull Request Operations": pr_test_success,
        "Tag Management": tag_test_success,
    }
    
    print("\n" + "="*50)
    print("GITHUB INTEGRATION TEST SUMMARY")
    print("="*50)
    
    for test_name, result in summary.items():
        status = "✅ PASS" if result else "❌ FAIL"
        print(f"{test_name:<25} {status}")
    
    total_tests = len(summary)
    passed_tests = sum(summary.values())
    print(f"\nTotal Tests: {total_tests}")
    print(f"Passed: {passed_tests}")
    print(f"Failed: {total_tests - passed_tests}")
    print(f"Success Rate: {(passed_tests/total_tests)*100:.1f}%")

# Display summary
display_test_summary()